In [1]:
import pandas as pd
import os
import numpy as np

In [4]:
#specify num models
num_models = 7
model_dir = "submissions"


models = os.listdir(f"{model_dir}")
models = sorted(models)
models

['submission_MODERNRES_seed0.csv',
 'submission_MODERNRES_seed1.csv',
 'submission_MODERNRES_seed2.csv',
 'submission_MODERNRES_seed3.csv',
 'submission_MODERNRES_seed4.csv',
 'submission_MODERNRES_seed5.csv',
 'submission_MODERNRES_seed6.csv']

Store all predictions in one dataframe

In [5]:
# combine model predictions in a single df
for i in range(0, num_models):
    curr_csv = models[i]

    # makes sure that column names identify the model
    model_name = curr_csv.split('.')[0]
    temp = pd.read_csv(f"{model_dir}/{curr_csv}")
    temp.set_index('id', inplace=True)
    temp.rename(columns={"label": f"label_{model_name}"}, inplace=True)
    if i == 0:
        combined = temp
    else:
        combined = pd.concat([combined, temp], axis=1)
combined

,label_submission_MODERNRES_seed0,label_submission_MODERNRES_seed1,label_submission_MODERNRES_seed2,label_submission_MODERNRES_seed3,label_submission_MODERNRES_seed4,label_submission_MODERNRES_seed5,label_submission_MODERNRES_seed6
id,,,,,,,
1,67,69,67,67,69,67,67
2,118,174,133,173,183,133,43
3,79,81,112,79,79,78,15
4,12,28,12,12,12,12,47
5,73,55,32,157,32,32,161
...,...,...,...,...,...,...,...
3996,57,152,181,19,168,181,181
3997,20,88,169,88,20,20,20
3998,103,103,103,130,85,196,131


Select mode

In [6]:
# function to select random mode in case of tie
def random_mode(row):
    modes = row.mode()
    return np.random.choice(modes)

# determine the mode across model predictions
combined['label'] = combined.apply(random_mode, axis=1).astype(int)
combined

,label_submission_MODERNRES_seed0,label_submission_MODERNRES_seed1,label_submission_MODERNRES_seed2,label_submission_MODERNRES_seed3,label_submission_MODERNRES_seed4,label_submission_MODERNRES_seed5,label_submission_MODERNRES_seed6,label
id,,,,,,,,
1,67,69,67,67,69,67,67,67
2,118,174,133,173,183,133,43,133
3,79,81,112,79,79,78,15,79
4,12,28,12,12,12,12,47,12
5,73,55,32,157,32,32,161,32
...,...,...,...,...,...,...,...,...
3996,57,152,181,19,168,181,181,181
3997,20,88,169,88,20,20,20,20
3998,103,103,103,130,85,196,131,103


Store results in two ways

In [ ]:
# store result, which contains all model predictions and the final label
combined.to_csv(f"/submission/all_pred.csv")

# drop columns of individual model predictions
for i in range(0, num_models):
    curr_csv = models[i]

    # makes sure that column names identify the model
    model_name = curr_csv.split('.')[0]
    combined.drop(columns=[f"label_{model_name}"], inplace=True)

# store result, which is final prediction after ensembling
combined.to_csv(f"/submission/ensemble_submission.csv")

OSError: Cannot save file into a non-existent directory: 'submission'

## Diff method

In [ ]:
import torch
import torch.nn as nn

class EnsembleModel(nn.Module):   
    def __init__(self, modelList):
        super().__init__()
        self.models = nn.ModuleList(modelList)
        self.classifier = nn.Linear(200 * len(modelList), 200)
        
    def forward(self, x):
        outputs = []
        for model in self.models:
            outputs.append(model(x))
        
        x_cat = torch.cat(outputs, dim=1)
        out = self.classifier(x_cat)
        return out


# Load the models
models = [model1, model2, model3]
ensemble_model = EnsembleModel(models)

# freeze parameters of individual models
for param in ensemble_model.parameters():
    param.requires_grad = False

# unfreeze parameters of the classifier
for param in ensemble_model.classifier.parameters():
    param.requires_grad = True    


ensemble_model = ensemble_model.to(device)